# 3 - Data Preprocessing (Translation)

In [ ]:
import pandas as pd

!pip install googletrans==4.0.0rc1

from googletrans import Translator

import math

from tqdm import tqdm

import time

## Google colab

In [ ]:
# from google.colab import drive

# drive.mount("/content/drive/")

# %cd /content/drive/My Drive/Colab Notebooks/

## Step 1. Import data

In [ ]:
df = pd.read_csv("inputs/tweets_1.csv")

df["tweet_id"] = df["tweet_id"].astype(str)

## Step 2.

In [ ]:
translator = Translator()

translator.raise_Exception = True

def translation(x):
    return translator.translate(x, src = "tl", dest = "en").text

## Step 3.

In [ ]:
tweet_id = []

index = []

before = []

after = []

N = len(df)

# REPLACEABLE

start = 0

rate = 50

n = math.floor((N - start) / rate) + 1

key = " \n\n\n "

## Step 4. Translation

In [ ]:
for j in tqdm(range(n)):
    
    try:
        end = start + rate
        after.append(translation(key.join(df["content_4"][start:end])))
        
    except TypeError:
        end = start + rate - 25
        after.append(translation(key.join(df["content_4"][start:end])))
        
    except IndexError:
        end = N
        after.append(translation(key.join(df["content_4"][start:end])))
        
    tweet_id.append(key.join(df["tweet_id"][start:end]))
    
    index.append("{0}:{1}".format(start, end))
    
    before.append(key.join(df["content_4"][start:end]))
    
    start = end
    
    if j % 50 == 0:
        time.sleep(1)

## Step 5.

In [ ]:
df_translated = pd.DataFrame(data = {"tweet_id" : tweet_id, "index" : index, "before" : before, "after" : after})

df_translated["before_count"] = df_translated["before"].apply(lambda x: len(x.split(key.strip(" "))))

df_translated["after_count"] = df_translated["after"].apply(lambda x: len(x.split(key.strip(" "))))

df_translated["missed"] = df_translated["before_count"] - df_translated["after_count"]

# df_translated.to_csv("inputs/tweets_translated_n/tweets_translated_x.csv", index = False)

## Step 7. Concatenate multiple `tweets_translated` csv files into a single `tweets_translated` csv file

In [ ]:
n = 5

df_translated = pd.concat([pd.read_csv("inputs/tweets_translated_n/tweets_translated_{}.csv".format(i + 1)) for i in range(n)], ignore_index = True)

print(sum(df_translated.after_count))

print(sum(df_translated.before_count))

print(sum(df_translated.missed))

# df_translated.to_csv("inputs/tweets_translated_n/tweets_translated_n.csv", index = False)

## Step 8.

In [ ]:
# df_before

df_before = pd.read_csv("inputs/tweets_1.csv")

df_before["tweet_id"] = df_before["tweet_id"].astype(str)

# df_translated

df_translated = pd.read_csv("inputs/tweets_translated_n/tweets_translated_n.csv")

# df_temporary

df_temporary = pd.DataFrame()

df_temporary["tweet_id"] = [j.strip().lower() for i in df_translated["tweet_id"] for j in i.split("\n\n\n")]

df_temporary["content_5"] = [j.strip().lower() for i in df_translated["after"] for j in i.split("\n\n\n")]

# df_after

df_after = df_before.merge(df_temporary, how = "inner", on = "tweet_id")

# df_after.to_csv("inputs/tweets_2.csv", index = False)